In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

XOR Gate Realization using Backpropagation Neural Network

In [16]:
#Define the neural network

class NeuralNetwork:
    def __init__(self,layers,alpha=0.1):

        self.W=[] # Initialize weights
        self.layers=layers
        self.alpha=alpha
    
        #Set weithgs to a random value
        for i in np.arange(0,len(layers)-2):

            w=np.random.randn(layers[i]+1,layers[i+1]+1)
            self.W.append(w/np.sqrt(layers[i]))

        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))
	
    # construct and return a string that represents the network
    def __repr__(self):

        return "NeuralNetwork: {}".format(
			"-".join(str(l) for l in self.layers))               
		
    #Sigmoid function
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    def sigmoid_deriv(self,x):
        return x*(1-x)

    
    #Training function
    def fit(self,X,y,epochs=1000,displayUpdate=100):

        #insert a column of 1's as the last entry in the feature matrix
        #this trick allows us to treat the bias as a trainable parameter
        X=np.c_[X,np.ones((X.shape[0]))]

        #loop over the desired number of epochs
        for epoch in np.arange(0,epochs):

            #loop over individual data point and train
            for (x,target) in zip(X,y):
                self.fit_partial(x,target)
            
            #check to see if we should display a training update
            if epoch == 0 or (epoch + 1) % displayUpdate == 0: 
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={:.7f}".format(epoch + 1, loss))


    #backpropagation implementation
    def fit_partial(self,x,y):
        #construct a list of output activations for each layer

        A = [np.atleast_2d(x)]

        #Forward propagate
        for layer in np.arange(0,len(self.W)):

            net = A[layer].dot(self.W[layer])
            out= self.sigmoid(net)

            A.append(out)

        #Back propagation
        
        #compute error
        error=A[-1]-y
        D=[error*self.sigmoid_deriv(A[-1])]

        #Now implement chain rule
        for layer in np.arange(len(A)-2,0,-1):

            delta= D[-1].dot(self.W[layer].T)
            delta = delta * self.sigmoid_deriv(A[layer])
            D.append(delta)
        
        D = D[::-1] # reverse as we loop backwards

        #Update weights
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    #Prediction function
    def predict(self, X, addBias=True):

        p = np.atleast_2d(X)

        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]
        
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
        
        return p
    
    #Loss function
    def calculate_loss(self, X, targets):
        # make predictions for the input data points then compute
        # the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        # return the loss
        return loss

## XOR Gate Realization

In [18]:
#truth table
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [19]:
#define neural network and train it
nn = NeuralNetwork([2, 2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

[INFO] epoch=1, loss=0.5037561
[INFO] epoch=100, loss=0.4990359
[INFO] epoch=200, loss=0.4968438
[INFO] epoch=300, loss=0.4926766
[INFO] epoch=400, loss=0.4806505
[INFO] epoch=500, loss=0.4462782
[INFO] epoch=600, loss=0.3689504
[INFO] epoch=700, loss=0.2155669
[INFO] epoch=800, loss=0.0880848
[INFO] epoch=900, loss=0.0448995
[INFO] epoch=1000, loss=0.0282782
[INFO] epoch=1100, loss=0.0201201
[INFO] epoch=1200, loss=0.0154178
[INFO] epoch=1300, loss=0.0124058
[INFO] epoch=1400, loss=0.0103301
[INFO] epoch=1500, loss=0.0088216
[INFO] epoch=1600, loss=0.0076803
[INFO] epoch=1700, loss=0.0067891
[INFO] epoch=1800, loss=0.0060754
[INFO] epoch=1900, loss=0.0054921
[INFO] epoch=2000, loss=0.0050069
[INFO] epoch=2100, loss=0.0045975
[INFO] epoch=2200, loss=0.0042476
[INFO] epoch=2300, loss=0.0039455
[INFO] epoch=2400, loss=0.0036820
[INFO] epoch=2500, loss=0.0034504
[INFO] epoch=2600, loss=0.0032453
[INFO] epoch=2700, loss=0.0030624
[INFO] epoch=2800, loss=0.0028984
[INFO] epoch=2900, loss=0.

In [20]:
# Run prediction
for (x, target) in zip(X, y):
	# make a prediction on the data point and display the result
	# to our console
	pred = nn.predict(x)[0][0]
	step = 1 if pred > 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.0130, step=0
[INFO] data=[0 1], ground-truth=1, pred=0.9869, step=1
[INFO] data=[1 0], ground-truth=1, pred=0.9890, step=1
[INFO] data=[1 1], ground-truth=0, pred=0.0082, step=0
